In [2]:
#import pytorch_lightning as pl
import lightning.pytorch as pl

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import wandb
from pytorch_lightning.loggers import WandbLogger

ModuleNotFoundError: No module named 'lightning'

In [2]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# Define the Lightning module
class MNISTLightning(pl.LightningModule):
    def __init__(self):
        super(MNISTLightning, self).__init__()
        self.model = MLP()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
        mnist_train = datasets.MNIST(root="../data", train=True, download=True, transform=transform)
        return torch.utils.data.DataLoader(mnist_train, batch_size=32, shuffle=True)

In [5]:
# Initialize wandb
wandb.init(project='mnist_mlp')
settings=wandb.Settings(silent="True")

# Create the WandbLogger
wandb_logger = WandbLogger()

# Train the model
trainer = pl.Trainer(max_epochs=5, logger=wandb_logger)
model = MNISTLightning()
trainer.fit(model)

# Close wandb run
wandb.finish()

/users/PLS0129/ysu0053/.conda/envs/graphgps/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | MLP  | 535 K 
-------------------------------
535 K     Trainable params
0         Non-trainable params
535 K     Total params
2.143     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.
/users/PLS0129/ysu0053/.conda/envs/graphgps/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, 

Epoch 4: 100%|██████████| 1875/1875 [00:21<00:00, 88.57it/s, v_num=tadp]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1875/1875 [00:21<00:00, 88.27it/s, v_num=tadp]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
train_loss,█▃▄▃▃▂▃▃▁▃▃▂▃▂▃▂▂▁▃▁▁▁▁▄▃▁▄▁▁▂▁▁▂▁▄▂▁▁▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,4
train_loss,0.15414
trainer/global_step,9349
